# Completed with Mapping Logic in Customer data 

In [18]:
#import spark as sp
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

In [19]:
spark = SparkSession\
    .builder\
    .appName("capstone_analysis")\
    .config("spark.jars","/Users/roy/Downloads/mysql-connector-j-8.0.32/mysql-connector-j-8.0.32.jar")\
    .getOrCreate()
spark = SparkSession.builder.appName("cdw_sapp_customer.json").getOrCreate()

23/02/28 10:50:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/02/28 10:50:17 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [20]:
customer_df = spark.read.format("json") \
    .option("header", True) \
    .load("cdw_sapp_custmer.json")

In [21]:
customer_df.show(1)

customer_df.printSchema()

+------+----------------+---------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+---------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|  Natchez|United States|AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
+------+----------------+---------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
only showing top 1 row

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_N

Transformation -  Mapping Logics

In [22]:
 #Convert the First Name 
customer_df = customer_df.withColumn('FIRST_NAME', initcap(customer_df.FIRST_NAME))

In [23]:
#Middle name
customer_df = customer_df.withColumn('MIDDLE_NAME', lower(customer_df.MIDDLE_NAME))

In [24]:
#LAST name
customer_df = customer_df.withColumn('MIDDLE_NAME', lower(customer_df.MIDDLE_NAME))

In [25]:
customer_df['FIRST_NAME','MIDDLE_NAME','LAST_NAME' ].show(3)

+----------+-----------+---------+
|FIRST_NAME|MIDDLE_NAME|LAST_NAME|
+----------+-----------+---------+
|      Alec|         wm|   Hooper|
|      Etta|    brendan|   Holman|
|    Wilber|   ezequiel|   Dunham|
+----------+-----------+---------+
only showing top 3 rows



In [26]:
# Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment)
customer_df = customer_df.withColumn("FULL_STREET_ADDRESS", concat_ws(", ", customer_df["Street_NAME"], customer_df["APT_NO"]))
    #'STREET_NAME', concat_ws(", ", "Street_NAME", "APT_NO"))

customer_df["FULL_STREET_ADDRESS","Street_NAME","APT_NO" ].show(3)

+--------------------+-----------------+------+
| FULL_STREET_ADDRESS|      Street_NAME|APT_NO|
+--------------------+-----------------+------+
|Main Street North...|Main Street North|   656|
|  Redwood Drive, 829|    Redwood Drive|   829|
|12th Street East,...| 12th Street East|   683|
+--------------------+-----------------+------+
only showing top 3 rows



In [27]:
customer_df = customer_df.withColumn('CUST_PHONE', customer_df['CUST_PHONE'].cast('string'))
customer_df.printSchema()


root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: string (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)
 |-- FULL_STREET_ADDRESS: string (nullable = false)



In [28]:
#Add the area code then change the format of phone number to (XXX)XXX-XXXX
customer_df = customer_df.withColumn('CUST_PHONE', concat(lit('(880)'), customer_df['CUST_PHONE'].substr(1,3), lit('-'), customer_df['CUST_PHONE'].substr(4,4)))
customer_df.show(2)

+------+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME| FULL_STREET_ADDRESS|
+------+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|(880)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North|Main Street North...|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|(880)123-8933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|

In [29]:
customer_df = customer_df.drop('APT_NO', 'STREET_NAME')
customer_df.show(2)

+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+------------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|     Natchez|United States|AHooper@example.com|(880)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North...|
|4210653310102868|Wethersfield|United States|EHolman@example.com|(880)123-8933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|  Redwood Drive, 829|
+----------------+------------+-------------+-------------------+----------

In [31]:
customer_df.write.format("jdbc") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
    .option("dbtable", "creditcard_capstone.cus_credit") \
          .option("user", "root") \
            .option("password", "password") \
            .mode("overwrite")\
            .save()
customer_df.show(1)

+----------------+---------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|  CREDIT_CARD_NO|CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|   CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN| FULL_STREET_ADDRESS|
+----------------+---------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
|4210653310061055|  Natchez|United States|AHooper@example.com|(880)123-7818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North...|
+----------------+---------+-------------+-------------------+-------------+----------+--------+----------+---------+--------------------+-----------+---------+--------------------+
only showing top 1 row

